<div style="background-color:blue; border-radius:2px; border:#000000 solid; padding: 15px; font-size:100%; text-align:center">
    <h1 align="center" style="color:#ffffff;"><b>Traffic Sign Recognition 🚦🔍</b></h1>
</div>

<div style="background-color:#000000; border-radius:2px; border:#000000 solid; padding: 15px; font-size:100%; text-align:center">
    <img src="https://media.tenor.com/NxZUCukvV5MAAAAd/traffic-cars.gif" alt="Animated GIF">
</div>

<div style="background-color:blue; border-radius:2px; border:#000000 solid; padding: 15px; font-size:100%; text-align:center">
    <h2 align="center" style="color:#ffffff;"><b>IMPORT DEPENDENCIES</b></h2>
</div>

In [3]:
!pip install -q numpy pandas opencv-python matplotlib seaborn tensorflow keras pillow plotly

In [7]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from PIL import Image
from tensorflow.keras.optimizers import Adam
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score
from matplotlib import style
import warnings
warnings.filterwarnings("ignore")
np.random.seed(42)

In [8]:
def read_images(directory_path):
    images = []
    labels = []

    valid_extensions = ('.jpg', '.jpeg', '.png')

    subdirectories = [subdir for subdir in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, subdir))]

    for subdir in subdirectories:
        subdirectory_path = os.path.join(directory_path, subdir)
        if not os.listdir(subdirectory_path):
            continue

        for filename in os.listdir(subdirectory_path):
            if filename.lower().endswith(valid_extensions):
                image_path = os.path.join(subdirectory_path, filename)
                image = cv2.imread(image_path)
                image = cv2.resize(image, (30, 30))
                images.append(image)

                label = int(subdir)
                labels.append(label)

    data = np.array(list(zip(images, labels)))
    
    return data

<div style="background-color:blue; border-radius:2px; border:#000000 solid; padding: 15px; font-size:100%; text-align:center">
    <h2 align="center" style="color:#ffffff;"><b>DATA PREPARETION</b></h2>
</div>

In [6]:
train_data = read_images('/kaggle/input/gtsrb-german-traffic-sign/Train')

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (39209, 2) + inhomogeneous part.

In [ ]:
print(f"Train Data Shape : {train_data.shape}")

In [ ]:
# Label Overview
classes = { 
    0:'Speed limit (20km/h)',
    1:'Speed limit (30km/h)', 
    2:'Speed limit (50km/h)', 
    3:'Speed limit (60km/h)', 
    4:'Speed limit (70km/h)', 
    5:'Speed limit (80km/h)', 
    6:'End of speed limit (80km/h)', 
    7:'Speed limit (100km/h)', 
    8:'Speed limit (120km/h)', 
    9:'No passing', 
    10:'No passing veh over 3.5 tons', 
    11:'Right-of-way at intersection', 
    12:'Priority road', 
    13:'Yield', 
    14:'Stop', 
    15:'No vehicles', 
    16:'Veh > 3.5 tons prohibited', 
    17:'No entry', 
    18:'General caution', 
    19:'Dangerous curve left', 
    20:'Dangerous curve right', 
    21:'Double curve', 
    22:'Bumpy road', 
    23:'Slippery road', 
    24:'Road narrows on the right', 
    25:'Road work', 
    26:'Traffic signals', 
    27:'Pedestrians', 
    28:'Children crossing', 
    29:'Bicycles crossing', 
    30:'Beware of ice/snow',
    31:'Wild animals crossing', 
    32:'End speed + passing limits', 
    33:'Turn right ahead', 
    34:'Turn left ahead', 
    35:'Ahead only', 
    36:'Go straight or right', 
    37:'Go straight or left', 
    38:'Keep right', 
    39:'Keep left', 
    40:'Roundabout mandatory', 
    41:'End of no passing', 
    42:'End no passing veh > 3.5 tons'
}

In [ ]:
labels = train_data[:, 1].astype(int)  # Extract the labels from the train_data array
unique_labels, label_counts = np.unique(labels, return_counts=True)
count_dict= {label: count for label, count in zip(unique_labels, label_counts)}
print(count_dict)

In [ ]:
base_color = '#8A2BE2'

color_scale = [[0, base_color], [1, 'rgb(30,30,30)']]

fig = go.Figure(data=[go.Bar(
    x=label_counts,
    y=[classes[label] for label in unique_labels],
    orientation='h',
    marker=dict(
        color=label_counts,
        coloraxis='coloraxis'
    )
)])

fig.update_layout(
    title='Label Counts in Train Data',
    xaxis_title='Count',
    yaxis_title='Label',
    plot_bgcolor='rgb(30,30,30)',
    paper_bgcolor='rgb(30,30,30)',
    font=dict(color='white'),
    coloraxis=dict(colorscale=color_scale)
)

fig.show()

In [ ]:
base_color = '#8A2BE2'
color_palette = sns.color_palette("husl", len(count_dict)).as_hex()

fig = go.Figure(data=[go.Pie(
    labels=list(count_dict.keys()),
    values=list(count_dict.values()),
    marker=dict(colors=color_palette),
)])

fig.update_layout(
    title='Label Distribution',
    font=dict(color='white'),
    paper_bgcolor='rgb(30,30,30)',
    plot_bgcolor='rgb(30,30,30)'
)

fig.show()

In [ ]:
indices = np.random.choice(train_data.shape[0], size=10, replace=False)
samples = train_data[indices]

fig = make_subplots(rows=2, cols=5, subplot_titles=[classes[label] for _, label in samples])

for i, (image, label) in enumerate(samples):
    row = i // 5 + 1
    col = i % 5 + 1

    fig.add_trace(
        go.Image(z=image),
        row=row,
        col=col
    )

fig.update_layout(
    title='Random Samples',
    font=dict(color='white'),
    paper_bgcolor='rgb(30,30,30)',
    plot_bgcolor='rgb(30,30,30)'
)

fig.show()

In [ ]:
train_ = pd.DataFrame(train_data, columns=['Image', 'Label'])
train_.head()

In [ ]:
# Specify the desired number of samples to select from each class
class_count = 200  

# Group the DataFrame by the 'Label' column
grouped = train_.groupby('Label')

# Use the apply function to randomly select the specified number of samples from each class
balanced_df = grouped.apply(lambda x: x.sample(class_count))

# Reset the index of the balanced DataFrame to maintain a clean structure
balanced_df.reset_index(drop=True, inplace=True)

In [ ]:
balanced_df.shape

In [ ]:
train_balanced = balanced_df.values

In [ ]:
new_labels = train_balanced[:, 1].astype(int)  # Extract the labels from the train_data array
new_unique_labels, new_label_counts = np.unique(new_labels, return_counts=True)
new_count_dict= {label: count for label, count in zip(new_unique_labels, new_label_counts)}
print(new_count_dict)

In [ ]:
base_color = '#8A2BE2'
color_palette = sns.color_palette("husl", len(new_count_dict)).as_hex()

fig = go.Figure(data=[go.Pie(
    labels=list(new_count_dict.keys()),
    values=list(new_count_dict.values()),
    marker=dict(colors=color_palette),
)])

fig.update_layout(
    title='Label Distribution',
    font=dict(color='white'),
    paper_bgcolor='rgb(30,30,30)',
    plot_bgcolor='rgb(30,30,30)'
)

fig.show()

In [ ]:
shuffled_train = train_balanced.copy()     # Create a copy of train_data to avoid modifying the original array
shuffled_train = np.array(shuffled_train)  # Shuffle the array randomly

In [ ]:
# Split the shuffled_train array into training and testing sets
train_set, test_set = train_test_split(shuffled_train, test_size=0.2, random_state=42)

# Separate the input (X) and output/label (y) arrays from the training set
x_train, y_train = train_set[:, 0], train_set[:, 1]

# Separate the input (X) and output/label (y) arrays from the testing set
x_test, y_test = test_set[:, 0], test_set[:, 1]

# Convert the data type of the arrays to int
x_train, y_train, x_test, y_test = map(np.array, [x_train, y_train, x_test, y_test])

# Normalize the input data
x_train, x_test = x_train / 255.0, x_test / 255.0

# Display the shapes of the arrays
print("x_train shape:", x_train.shape)
print("x_test  shape:", x_test.shape)
print("y_train shape:", y_train.shape)
print("y_test  shape:", y_test.shape)

In [ ]:
# Convert target labels to one-hot encoding
y_train = to_categorical(y_train, num_classes=43)
y_test  = to_categorical(y_test, num_classes=43)

<div style="background-color:blue; border-radius:2px; border:#000000 solid; padding: 15px; font-size:100%; text-align:center">
    <h2 align="center" style="color:#ffffff;"><b>MODEL ENGINEERING</b></h2>
</div>

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu', input_shape=(30,30,3)),
    tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    tf.keras.layers.BatchNormalization(axis=-1),
    
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    tf.keras.layers.BatchNormalization(axis=-1),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(rate=0.5),
    
    tf.keras.layers.Dense(43, activation='softmax')
])

In [ ]:
lr = 0.001
epochs = 30
opt = Adam(learning_rate=lr)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [ ]:
x_train = np.array(x_train)
y_train = np.array(y_train)

aug = ImageDataGenerator(
    rotation_range=15,
    zoom_range=0.2,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode="reflect"
)

history = model.fit(aug.flow(x_train, y_train, batch_size=32), epochs=epochs, validation_data=(x_test, y_test))

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=list(range(1, epochs+1)),
    y=history.history['loss'],
    mode='lines',
    name='Training Loss'
))

fig.add_trace(go.Scatter(
    x=list(range(1, epochs+1)),
    y=history.history['val_loss'],
    mode='lines',
    name='Validation Loss'
))

fig.update_layout(
    title='Model Loss',
    xaxis_title='Epochs',
    yaxis_title='Loss',
    font=dict(color='white'),
    paper_bgcolor='rgb(30,30,30)',
    plot_bgcolor='rgb(30,30,30)'
)

fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=list(range(1, epochs+1)),
    y=history.history['accuracy'],
    mode='lines',
    name='Training Accuracy'
))

fig.add_trace(go.Scatter(
    x=list(range(1, epochs+1)),
    y=history.history['val_accuracy'],
    mode='lines',
    name='Validation Accuracy'
))

fig.update_layout(
    title='Model Accuracy',
    xaxis_title='Epochs',
    yaxis_title='Accuracy',
    font=dict(color='white'),
    paper_bgcolor='rgb(30,30,30)',
    plot_bgcolor='rgb(30,30,30)'
)

fig.show()

In [ ]:
# Convert NumPy array to DataFrame
shuffled_train_dataframe = pd.DataFrame(shuffled_train)
# Save DataFrame as CSV file
shuffled_train_dataframe.to_csv('data.csv', index=False)

In [ ]:
model.save("saved_model.h5")

In [ ]:
def predict_image(model_path, image_path):
    model = tf.keras.models.load_model(model_path)
    image = cv2.imread(image_path)
    image = cv2.resize(image, (30, 30))
    image = np.expand_dims(image, axis=0)
    image = image / 255.0
    predictions = model.predict(image)
    predicted_label = np.argmax(predictions)
    
    return predicted_label

model_path = "/kaggle/working/saved_model.h5"
image_path = "/kaggle/input/gtsrb-german-traffic-sign/Test/00080.png"
predicted_label = predict_image(model_path, image_path)
print("Predicted Label:", classes[predicted_label])

In [ ]:
#  Let's practice object-oriented programming (OOP) with TensorFlow :) 
"""
class TrafficSignModel:
    def __init__(self):
        self.model = self.build_model()
        self.lr = 0.001
        self.epochs = 30
        self.opt = Adam(learning_rate=self.lr)
        self.model.compile(loss='categorical_crossentropy', optimizer=self.opt, metrics=['accuracy'])

    def build_model(self):
        model = tf.keras.Sequential([
            Conv2D(filters=16, kernel_size=(3, 3), activation='relu', input_shape=(30,30,3)),
            Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
            MaxPool2D(pool_size=(2, 2)),
            BatchNormalization(axis=-1),
            Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
            Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
            MaxPool2D(pool_size=(2, 2)),
            BatchNormalization(axis=-1),
            Flatten(),
            Dense(512, activation='relu'),
            BatchNormalization(),
            Dropout(rate=0.5),
            Dense(43, activation='softmax')
        ])
        return model

    def train_model(self, x_train, y_train, x_test, y_test):
        aug = ImageDataGenerator(
            rotation_range=15,
            zoom_range=0.2,
            width_shift_range=0.15,
            height_shift_range=0.15,
            shear_range=0.2,
            horizontal_flip=True,
            vertical_flip=True,
            fill_mode="reflect"
        )
        history = self.model.fit(aug.flow(x_train, y_train, batch_size=32), epochs=self.epochs, validation_data=(x_test, y_test))
        return history

    def plot_loss(self, history):
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=list(range(1, self.epochs+1)), y=history.history['loss'], mode='lines', name='Training Loss'))
        fig.add_trace(go.Scatter(x=list(range(1, self.epochs+1)), y=history.history['val_loss'], mode='lines', name='Validation Loss'))
        self.update_plot_layout(fig)
        fig.show()

    def plot_accuracy(self, history):
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=list(range(1, self.epochs+1)), y=history.history['accuracy'], mode='lines', name='Training Accuracy'))
        fig.add_trace(go.Scatter(x=list(range(1, self.epochs+1)), y=history.history['val_accuracy'], mode='lines', name='Validation Accuracy'))
        self.update_plot_layout(fig)
        fig.show()

    def update_plot_layout(self, fig):
        fig.update_layout(
            title='Model Loss/Accuracy',
            xaxis_title='Epochs',
            font=dict(color='white'),
            paper_bgcolor='rgb(30,30,30)',
            plot_bgcolor='rgb(30,30,30)'
        )

    def save_dataframe_to_csv(self, data, filename='data.csv'):
        shuffled_train_dataframe = pd.DataFrame(data)
        shuffled_train_dataframe.to_csv(filename, index=False)

    def save_model(self, filename='saved_model.h5'):
        self.model.save(filename)

    def predict_image(self, model_path, image_path):
        model = tf.keras.models.load_model(model_path)
        image = cv2.imread(image_path)
        image = cv2.resize(image, (30, 30))
        image = np.expand_dims(image, axis=0)
        image = image / 255.0
        predictions = model.predict(image)
        predicted_label = np.argmax(predictions)
        return predicted_label

if __name__ == "__main__":
    # Sample usage
    traffic_sign_model = TrafficSignModel()
    # Train the model with your actual data
    history = traffic_sign_model.train_model(x_train, y_train, x_test, y_test)
    # Plot loss and accuracy
    traffic_sign_model.plot_loss(history)
    traffic_sign_model.plot_accuracy(history)
    # Save data to CSV and model to h5
    traffic_sign_model.save_dataframe_to_csv(shuffled_train)
    traffic_sign_model.save_model()
    # Predict image label
    model_path = "saved_model.h5"
    image_path = "input/gtsrb-german-traffic-sign/Test/00080.png"
    predicted_label = traffic_sign_model.predict_image(model_path, image_path)
    print("Predicted Label:", classes[predicted_label])

"""